<a href="https://colab.research.google.com/github/maorgreshler/NN_google_colab/blob/main/NN_shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import joblib
import os

#os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_size)
        )
    def forward(self, x):
        return self.model(x)

# Training function with validation and TensorBoard logging
def train_model(df, input_columns, output_columns, num_epochs=20, batch_size=64, learning_rate=0.001,model_name):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    input_data = df[input_columns].values
    output_data = df[output_columns].values
    input_data_scaled = StandardScaler().fit_transform(input_data)
    output_data_scaled = StandardScaler().fit_transform(output_data)

    # Split the data into train and validation sets
    X_train, X_test, y_train, y_test = train_test_split(input_data_scaled, output_data_scaled, test_size=0.2, random_state=42)

    # Convert data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

    # Create datasets and dataloaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    # Data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


    # Initialize model, loss function, and optimizer
    model = MLP(input_size=len(input_columns), output_size=len(output_columns)).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


    # TensorBoard writer
    #writer = SummaryWriter()

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

        # Validation loop
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs = inputs.to(device)
                targets = targets.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)

        # Calculate average losses
        train_loss /= train_size
        val_loss /= val_size

        # Log to TensorBoard
        writer.add_scalars('Loss', {'Train': train_loss, 'Validation': val_loss}, epoch)


        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

    writer.close()
    return model, (full_dataset.input_scaler, full_dataset.target_scaler)

# Main execution
if __name__ == '__main__':
    df = pd.read_csv('overfit_test.csv') # Load the csv to "files" in the left up corner of google colab
    input_columns = ['velocity', 'yaw_angle', 'yaw_rate', 'slip_angle', 'front_wheel',
                     'back_wheel', 'acceleration', 'angular_acceleration']  # D to K
    output_columns = ['delta_x_position', 'delta_y_position', 'delta_theta', 'delta_velocity', 'delta_yaw_angle',
                      'delta_yaw_rate', 'delta_slip_angle', 'delta_front_wheel', 'delta_back_wheel']  # L to T
    model, scalers = train_model(df, input_columns, output_columns,
                                 num_epochs=20, batch_size=64, learning_rate=0.001, model_name="TheBestNNInTheWest.pth") # Don't forget change the hidden layers!

    #os.chdir(r'C:\Users\matan\OneDrive\Desktop\Matan\project A\ProjectA_col\NN')#change acrodinly
    #torch.save(model.state_dict(), model_name)
    #print("Script executed in:", os.getcwd())
